In [1]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv(r'training_set-411.csv')
#data = data.drop(columns=['electrons'])
X = data.iloc[:, 0:-5] 
y = data.iloc[:,-5:]
# X.replace('none', np.nan, inplace=True)
symbol_columns = ['base', 'TM', 'modifications_ring', 'modifications_chain', 'Conjugate_structure','period','group_id']
for column in symbol_columns:
    X[column], _ = pd.factorize(X[column])

for column in X.keys():
    X[column] = (X[column] - X[column].mean()) / X[column].std()
# imputer = SimpleImputer(strategy='mean')
# data_filled = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

In [4]:
testdata = pd.read_csv(r'all_structure.csv')
# valdata = valdata.drop(columns=['num','KPOINTS','E-opt'])
# X.replace('none', np.nan, inplace=True)
testdata = testdata[testdata['traning_data'].isnull()]
result = testdata
symbol_columns = ['base', 'TM', 'modifications_ring', 'modifications_chain', 'Conjugate_structure','period','group_id']
testdata = testdata.drop(columns=['num','KPOINTS','E-opt', 'traning_data','name'])
testdata = testdata.dropna()
for column in symbol_columns:
    testdata[column], _ = pd.factorize(testdata[column])
for column in testdata.keys():
    testdata[column] = (testdata[column] - testdata[column].mean()) / testdata[column].std()

#r1<-0.4~r2;
#r3<-0.4~r4;
#r2|r4<0.4~r5

In [5]:
for i in range(5):
    print(y.columns[i], "as label")
    data1 = pd.concat([X, y.iloc[:,i]], axis=1)
    # 删除含有缺失标签的样本
    data_cleaned = data1.dropna(subset=[data1.columns[-1]])
    # 分离特征和标签
    X_cleaned = data_cleaned.iloc[:, :-1]  # 取前n-1列为特征
    y_cleaned = data_cleaned.iloc[:,-1]  # 取最后一列为标签
    X_train, X_val, y_train, y_val = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)
    # 初始化梯度提升回归模型
    # gb = RandomForestRegressor(max_depth = 20, min_samples_leaf = 1, min_samples_split = 2, n_estimators = 50)
    #{'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
    
    gb = GradientBoostingRegressor(learning_rate = 0.1, n_estimators = 50)  
    #0{'learning_rate': 0.1, 'n_estimators': 50}
    #1{'learning_rate': 0.01, 'n_estimators': 200}
    #3{'learning_rate': 0.1, 'n_estimators': 50}
    #{'learning_rate': 0.1, 'n_estimators': 50}

    # 拟合模型
    gb.fit(X_train, y_train)
    # 在测试集上做预测
    y_pred = gb.predict(X_val)
    # 评估模型性能
    mse = mean_squared_error(y_val, y_pred)
    testvalue = gb.predict(testdata)
    result[y.columns[i]] = testvalue
    print("Gradient Boosting Regression Mean Squared Error: ", mse)

ΔEN2(end-on) as label
Gradient Boosting Regression Mean Squared Error:  0.028953094127172226
ΔGN2H(end-on) as label
Gradient Boosting Regression Mean Squared Error:  0.024707082433029007
ΔEN2(side-on) as label
Gradient Boosting Regression Mean Squared Error:  0.05615984038977376
ΔGN2H(side-on) as label
Gradient Boosting Regression Mean Squared Error:  0.0591921868814992
ΔGNH3 as label
Gradient Boosting Regression Mean Squared Error:  0.03268849263087953


In [6]:
result['Flag'] = 0
filtered_df = result[(result.iloc[:,-5].astype(float) < -0.4) & (result.iloc[:,-3].astype(float) < -0.4) & ((result.iloc[:,-4].astype(float) < 0.4) | (result.iloc[:,-2].astype(float) < 0.4))]
result.loc[(result.iloc[:,-5].astype(float) < -0.4) & (result.iloc[:,-3].astype(float) < -0.4) & ((result.iloc[:,-4].astype(float) < 0.4) | (result.iloc[:,-2].astype(float) < 0.4)), 'Flag'] = 1

In [7]:
result.to_csv('result.csv', index=False)